# Codificación con llave pública RSA

Los servicios de inteligencia han interceptado el siguiente WhatsApp, que ha sido enviado por el cabecilla de un grupo al que están investigando.

<span style="color:blue">
Hola a todos,
entrar en:
$$963165702406078610933268087093959165865367524389470937341450164058799873046090241902268240522430399039842519010966073984353131388865535909155540264424417892621L$$
</span>

Sabemos, gracias a un infiltrado, al que llamaremos Pepe, para proteger su identidad, que ese número corresponde a la codificación (usando el protocolo RSA) de la dirección de una web.  


De las llaves públicas e y n que se han usado para codificar la dirección de la web. Pepe nos ha dicho que el módulo $n$ es 
$$
10000000000000000000000000000000000000000000000000000000000000000000033000000000000000002890000000000000000000000000000000000000000000000000000000000000000009537L
$$

Del que además sabemos que es el producto de dos primos uno de ellos muy cercano a $10^{90}$. 

Acerca del exponente $e$, que se ha usado, solo sabemos que está entre $12334$ y $12345$. 

En esa web se habla de un famoso detective cuyo apellido ha sido usado como llave para codificar, siguiendo el procedimiento de Vigenère, una clave. La matriz de Vigenère que se ha usado se obtiene mediante una traslación de 6 lugares de las 26 letras del alfabeto (sin Ñ), de manera que en el lugar 1,1 aparece la letra G.

Afortunadamente tenemos pinchado el WhatsApp del cabecilla e interceptamos este otro mensaje:

<span style="color:blue">
Lo vamos a petar compañer@s
os dejo las instru donde siempre
la clave para entrar es: PIJVHYJX
</span>

Pepe dice que esta banda ha pirateado la web de la asignatura TAN y Criptografía de cuarto curso de matemáticas de la UGR y que es en esta web donde dejan las instrucciones para cometer sus fechorías. Solo hay que introducir la clave correcta cuando se pulsa el botón etiquetado como "este botón" que aparece en esa web.

Puede ser una cuestión de seguridad nacional saber que está tramando este peligroso grupo. ¿Podrías ayudar a nuestro servicio de inteligencia averiguando que traman?

In [1]:
from sympy import *
from TANJCC import *

Partimos de las pistas dadas por el infiltrado: Como $n$ es producto de dos primos y uno de ellos es cercano a $10^{90}$, con $\texttt{nextprime}$ encontramos el siguiente número primo, $p$ que resulta que divide a $n$ y da como resultado otro primo, $q$. Por tanto, si obtenemos la clave pública $e$, podremos decodificar la dirección web dada.

In [2]:
n = 10000000000000000000000000000000000000000000000000000000000000000000033000000000000000002890000000000000000000000000000000000000000000000000000000000000000009537L
p = nextprime(10**90)
q = n / p
print p
print q
isprime(q)

1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000289
10000000000000000000000000000000000000000000000000000000000000000000033


True

Como $e \in [12334,12345]$, tendremos que buscar cuál de estos números es primo relativo de $n$, propiedad que debe cumplir:

In [3]:
for e in range(12334,12346):
    if( gcdex(e,(p-1)*(q-1))[2] == 1):
        print e

12335
12337
12341
12343


No podemos deducir de aquí cuál es la clave, luego debemos comprobar que la web que tenemos cifrada se corresponde con una dirección web. Comprobamos para cada $e$ en primer lugar que sea primo relativo con $\varphi(n)=(p-1)(q-1)$ por ser $n$ producto de dos primos. Obtenemos el inverso de $e$, $u$, y entonces usamos $x \equiv y^u \mod(n)$ para obtener la dirección web decodificada. Ahora bien, los números en hexadecimal que se corresponden con cadenas de texto siguiendo la codificación realizada con el método $\texttt{code('hex')}$ tienen longitud par, luego añadimos este filtro e imprimimos las cadenas resultantes, obteniendo únicamente la dirección que aparece a continuación:

In [4]:
web = 963165702406078610933268087093959165865367524389470937341450164058799873046090241902268240522430399039842519010966073984353131388865535909155540264424417892621L
for e in range(12334,12346):
    if( gcdex(e,(p-1)*(q-1))[2] == 1):
        u = gcdex(e,(p-1)*(q-1))[0] % ((p-1)*(q-1))
        x = pow(web,int(u),n)
        if(len(dectohex_rec(x)) % 2 == 0):
            print dectohex_rec(x).decode('hex')

https://es.wikipedia.org/wiki/Andrea_Camilleri


Este escritor y guionista es autor de una serie de novelas protagonizada por el Comisario Montalbano, en honor al escritor Vázquez Montalbán, también autor de novelas policíacas. Por tanto la clave de Vigenere será MONTALBANO.
En el método para decodificar una palabra siguiendo una clave, se toma en primer lugar la primera columna, donde iremos buscando las letras de la clave, y en cada fila correspondiente se va buscando sucesivamente las letras del mensaje codificado, añadiendo a una lista de letras que forman la palabra codificada la primera letra de cada columna, proceso inverso al necesario para codificar

In [5]:
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
tabla_vigenere = [ [LETTERS[(i+j+6)%26] for i in range(26) ] for j in range(26)]

clave_vigenere = 'MONTALBANO'
decode_vigenere(tabla_vigenere, 'PIJVHYJX', clave_vigenere)

JACINTOD


Con la clave JACINTOD, he accedido a la página donde seguían las instrucciones, encontrando un nuevo $n$, cuyos factores (primos) se pueden deducir fácilmente debido a la forma del número. Por tanto, obtenemos así la clave privada:

In [6]:
n=100000000190000000000000130000000247
p = 10000000019
print isprime(p)
q = n / p
print isprime(q)

True
True


Para ahorrar repetir el proceso anterior para cada palabra codificada del mensaje he implementado una función $\texttt{deco_rsa()}$ que automatice esta tarea. Simplemente habrá que elevar cada palabra al inverso de $e$, y al pasarlo a hexadecimal y de ahí a string, recuperar el mensaje original

In [7]:
e = 1211
mensaje=[62718023028666516652217553286447580L,7498947686663464945219055124624672L,5097532352808044988339897796965481L, 91589262635941722393701864307290432L,32253663039631038701991850668453454L,53773066331614972385019608760594210L, 44872556814103066346332116543585947L,927595488059769026187019360488764L,49395339185930770640825500218907604L, 82765863643386211968916760020252865L,25190253146559911376083367250316299L,81516543968865406348490251727544031L, 46973933837819443668504149601583430L,77960341753470006710619206199970261L,49395339185930770640825500218907604L, 79254408903157759140039430389457364L,32253663039631038701991850668453454L,69780185215451483837569951955699571L, 29181696113870199493001011506819598L,90746575521726863772619585452064895L,53321108584650112172697208811566958L, 89581607855380843846698601631339583L,4708233782086952953447867649890291L,81432953437104554702397572297116064L, 36748330705811761693354226961591564L,1738593413407962465957970386712872L,32253663039631038701991850668453454L, 6750255873197567500580868078789613L,3974641490716322416716012898882504L,69986814838734158265662071880367929L, 33080062031670887360059604610536909L,40912383938472947746699773999940668L,4708233782086952953447867649890291L, 60483336929846569313797889446933915L,50667049569681047447267330219362885L,16290171361242663103615946233640411L, 47438932232141836314849694319021353L,39517692056423301078064249847281553L,5059085049209552086870359812535240L, 62718023028666516652217553286447580L,34826601305744909271655808230506457L,32253663039631038701991850668453454L, 91586940261787857401151443814477655L,61306029703084185462695712502140576L,83961571813158583840345582468490739L, 53321108584650112172697208811566958L,16290171361242663103615946233640411L,33080062031670887360059604610536909L, 33769266318800097853303530008277240L,32253663039631038701991850668453454L,59548761183455377590539187724065107L, 40242014248379983165075976953067146L,39517692056423301078064249847281553L,29181696113870199493001011506819598L, 32435388911346208285687326957122563L,53321108584650112172697208811566958L,49395339185930770640825500218907604L, 20984102453995705653556910024287620L,23099420478417425445521464902274289L,32601348700812862397811686988230384L, 38217714717707056935821519501830833L,58757499046435065954253182867578106L,927595488059769026187019360488764L, 97463369160658535386970147458030133L,596227106103230803334107174826800L,81516543968865406348490251727544031L, 33080062031670887360059604610536909L,94139569715939059099802866251926914L,52678403584758883400564489432746954L, 32253663039631038701991850668453454L,44476434974326501460048387026732L,55680754582162596062709617863536289L, 62718023028666516652217553286447580L,3789955926090140936068306445042312L,32253663039631038701991850668453454L,66686506397057715816262021262989728L]
print deco_rsa(mensaje, e, p, q)

To be, or not to be: that is the question: Whether ‘tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take arms against a sea of troubles, And by opposing end them? To die: to sleep; No more; and by a sleep to say we end The heart-ache and the thousand natural shocks That flesh is heir to, ‘tis a consummation Devoutly to be wish’d. To die, to sleep.


Este soliloquio pertenece a la obra *Hamlet* de William Shakespeare